In [1]:
import requests
import base64
import numpy as np
from datetime import datetime
import bitstring

In [36]:
tile_url = """http://lcmap-test.cr.usgs.gov/landsat/tiles?\
ubid=LANDSAT_7/ETM/sr_band1&\
ubid=LANDSAT_7/ETM/sr_band2&\
ubid=LANDSAT_7/ETM/sr_band3&\
ubid=LANDSAT_7/ETM/sr_band4&\
ubid=LANDSAT_7/ETM/sr_band5&\
ubid=LANDSAT_7/ETM/toa_band6&\
ubid=LANDSAT_7/ETM/sr_band7&\
ubid=LANDSAT_7/ETM/cfmask&\
x=-2013585&y=3095805&acquired=1982-01-01/2017-01-01"""
fmask_url = """http://lcmap-test.cr.usgs.gov/landsat/tiles?ubid=LANDSAT_7/ETM/cfmask&x=-2013585&y=3095805&acquired=1982-01-01/2017-01-01"""
spec_url = "http://lcmap-test.cr.usgs.gov/landsat/tile-spec/LANDSAT_7/ETM/sr_band1"

In [37]:
tile_url

'http://lcmap-test.cr.usgs.gov/landsat/tiles?ubid=LANDSAT_7/ETM/sr_band1&ubid=LANDSAT_7/ETM/sr_band2&ubid=LANDSAT_7/ETM/sr_band3&ubid=LANDSAT_7/ETM/sr_band4&ubid=LANDSAT_7/ETM/sr_band5&ubid=LANDSAT_7/ETM/toa_band6&ubid=LANDSAT_7/ETM/sr_band7&ubid=LANDSAT_7/ETM/cfmask&x=-2013585&y=3095805&acquired=1982-01-01/2017-01-01'

In [4]:
fmask_url

'http://lcmap-test.cr.usgs.gov/landsat/tiles?ubid=LANDSAT_7/ETM/cfmask&x=-2013585&y=3095805&acquired=1982-01-01/2017-01-01'

In [5]:
def sort_band_data(band, field):
    return sorted(band, key=lambda x: x[field])

def b64_to_bytearray(data):
    return np.frombuffer(base64.b64decode(data), np.int16)

def dtstr_to_ordinal(dtstr):
    _dt = datetime.strptime(dtstr, '%Y-%m-%dT%H:%M:%SZ')
    return _dt.toordinal()

In [44]:
tile_resp = requests.get(tile_url)

In [45]:
tile_resp.status_code

200

In [10]:
#fmask_resp = requests.get(fmask_url)

In [11]:
#fmask_resp.status_code

200

In [136]:
band1, band2, band3, band4, band5, band6, band7, fmask = [], [], [], [], [], [], [], [] 

In [123]:
json_data = tile_resp.json()

In [124]:
# sort the data by band into their respective buckets
for item in json_data:
    which_band = item['ubid'][-5:]
    vars()[which_band].append(item)

In [125]:
for i in "band1 band2 band3 band4 band5 band6 band7 fmask".split(" "):
    print "{}: {}".format(i, len(vars()[i]))

band1: 296
band2: 296
band3: 296
band4: 296
band5: 296
band6: 296
band7: 296
fmask: 967


In [126]:
source_names = [i['source'] for i in band2]
fmask_names = [i['source'] for i in fmask]
valid_sources = set(source_names) & set(fmask_names)

In [127]:
len(valid_sources)

282

In [128]:
#bandx = band1
#buckets = (bandx, band2, band3, band4, band5, band6, band7, fmask)

In [129]:
for bucket in "band1 band2 band3 band4 band5 band6 band7 fmask".split(" "):
    _orig = vars()[bucket]
    _clean = vars()[bucket+'_clean'] = []
    for item in _orig:
        if item['source'] in valid_sources:
            _clean.append(item)
        #else:
            #print bucket,' rejected'


In [131]:
for i in "band1_clean band2_clean band3_clean band4_clean band5_clean band6_clean band7_clean fmask_clean".split(" "):
    print "{}: {}".format(i, len(vars()[i]))

band1_clean: 282
band2_clean: 282
band3_clean: 282
band4_clean: 282
band5_clean: 282
band6_clean: 282
band7_clean: 282
fmask_clean: 282


In [132]:
#for bucket in buckets:
for bucket in "band1 band2 band3 band4 band5 band6 band7 fmask".split(" "):
    _clean = vars()[bucket+'_clean']
    _sorted = vars()[bucket+'_sorted'] = sort_band_data(_clean, 'acquired')
    for item in _sorted:
        item['bytedata'] = b64_to_bytearray(item['data'])

In [135]:
len(band1_sorted[0]['bytedata'])

10000

In [138]:
#dtstr_to_ordinal(band1_sorted[0]['acquired'])
dates = [dtstr_to_ordinal(i['acquired']) for i in band2_sorted]

In [139]:
len(dates)

282

In [140]:
ubid_band_dict = {
    'tm': {'red': 'band3',
           'blue': 'band1',
           'green': 'band2',
           'nirs': 'band4',
           'swirs1': 'band5',
           'swirs2': 'band7',
           'thermals': 'band6',
           'qas': 'fmask'},
    'oli': {'red': 'band4',
            'blue': 'band2',
            'green': 'band3',
            'nirs': 'band5',
            'swirs1': 'band6',
            'swirs2': 'band7',
            'thermals': 'band10',
            'qas': 'fmask'}
}

In [141]:
mapping = ubid_band_dict['tm']

In [143]:
reds = vars()[mapping['red']+'_sorted']
greens = vars()[mapping['green']+'_sorted']
blues = vars()[mapping['blue']+'_sorted']
nirs = vars()[mapping['nirs']+'_sorted']
swirs1 = vars()[mapping['swirs1']+'_sorted']
swirs2 = vars()[mapping['swirs2']+'_sorted']
thermals = vars()[mapping['thermals']+'_sorted']
qas = vars()[mapping['qas']+'_sorted']

In [144]:
for grp in (reds, greens, blues, nirs, swirs1, swirs2, thermals, qas):
    print len(grp)

282
282
282
282
282
282
282
282


In [146]:
# assuming tile shape of 100x100, cell count of 10000
cell_count = 10000
tile_range = range(0, cell_count)
observation_range = range(0, len(band1_sorted))

In [223]:
# each bandx_sorted grouping of data convert to a two-dimensional numpy array
band1_array = np.array([b64_to_bytearray(i['data']) for i in band1_sorted])
band1_array

array([[ 516,  536,  476, ...,  422,  422,  438],
       [ 380,  370,  361, ...,  445,  537,  520],
       [2966, 2978, 3071, ..., 2579, 2450, 2624],
       ..., 
       [ 260,  243,  231, ...,  223,  263,  255],
       [ 977,  981,  973, ..., 1206, 1172, 1241],
       [1553, 1405, 1355, ..., 3324, 3343, 3517]], dtype=int16)

In [225]:
#>>> b[0:3,0:1]
# should be 281 tiles deep
band1_array[0:281,0:1]

array([[  516],
       [  380],
       [ 2966],
       [  184],
       [  260],
       [  972],
       [  855],
       [   89],
       [ 1766],
       [ 1115],
       [ 5163],
       [ 3674],
       [  424],
       [20000],
       [20000],
       [  368],
       [ 3024],
       [20000],
       [  422],
       [  837],
       [  313],
       [  214],
       [  205],
       [ 3035],
       [20000],
       [  225],
       [ 4533],
       [ 4644],
       [ 3010],
       [  380],
       [ 7458],
       [ 2359],
       [  519],
       [ 3263],
       [ 1315],
       [  465],
       [20000],
       [  417],
       [  303],
       [20000],
       [ 2874],
       [  183],
       [  343],
       [  262],
       [  998],
       [ 1087],
       [ 4096],
       [ 1994],
       [  309],
       [ 3256],
       [  329],
       [20000],
       [  472],
       [  215],
       [  305],
       [ 1203],
       [  388],
       [ 1864],
       [  286],
       [  226],
       [  197],
       [ 6287],
       [

In [217]:
aa = b64_to_bytearray(band1_sorted[0]['data'])
len(aa)
aa[0]

516

In [213]:
#z = bytearray(x)
z = np.frombuffer(x, np.int16)
len(z)

10000

In [203]:
import struct
#struct.calcsize(x)
struct.unpack_from('@iiii', x)

(35127812, 31326684, 28836314, 29884901)

In [107]:
spec_url = "http://lcmap-test.cr.usgs.gov/landsat/tile-spec/LANDSAT_7/ETM/sr_band1"

In [31]:
spec_resp = requests.get(spec_url)

In [32]:
# date needs to be iso8601.to_ordinal

[u'acquired', u'source', u'y', u'x', u'ubid', u'data']

In [138]:
tdata = b64_to_bytearray(band1_sorted[0]['data'])

In [142]:
len(tdata)

20000

In [145]:
tdata[10000]

148

In [33]:
tile_ba = bytearray(base64.b64decode(tile_resp.json()[0]['data']))

In [34]:
data_shape = spec_resp.json()['data_shape']

In [35]:
# always a square
tile_cell_count = data_shape[0] * data_shape[1]

In [36]:
tile_array = np.array(tile_ba)

In [37]:
num_tiles = len(tile_array) / tile_cell_count
num_tiles

2

In [38]:
split_array = np.reshape(tile_array, (num_tiles, -1))

In [39]:
split_array

array([[ 91,   4,  55, ...,  13,  66,  14],
       [ 39,  17, 100, ...,  21,  49,  22]], dtype=uint8)

In [40]:
len(split_array)


2

In [42]:
tile_range = range(0, len(split_array[0]))
tile_range[-1]

9999

In [ ]:
## numpy array indexing example
>>> b = np.arange(12).reshape(3,4)
>>> b
array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])
>>>
>>> b[0:3,0:1]
array([[0],
       [4],
       [8]])
>>> b[0:3,1:2]
array([[1],
       [5],
       [9]])
>>> b[0:3,2:3]
array([[ 2],
       [ 6],
       [10]])


In [ ]:
# results = ccd.detect(dates, reds, greens, blues, nirs, swir1s, swir2s, thermals, qas)
# need: date_array, reds_array, greens_array, blues_array, nirs_array, swirs2_array,...
# on a per cell basis